# Decoding Quantum Hypergraph Product Codes

In [2]:
import numpy as np
from tqdm import tqdm
import qecstruct as qc
import qecsim.paulitools as pt

import matplotlib
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib.ticker import FormatStrFormatter

from mdopt.mps.utils import marginalise, create_custom_product_state
from mdopt.contractor.contractor import mps_mpo_contract
from mdopt.optimiser.utils import (
    SWAP,
    COPY_LEFT,
    XOR_BULK,
    XOR_LEFT,
    XOR_RIGHT,
)
from examples.decoding.decoding import (
    apply_constraints,
    apply_bitflip_bias,
)
from examples.decoding.decoding import (
    decode_css,
    pauli_to_mps,
    css_code_checks,
    css_code_logicals,
    css_code_logicals_sites,
    css_code_constraint_sites,
)

In [3]:
import random

def generate_pauli_error_string(num_qubits, error_rate):
    """
    Generate a random Pauli error string based on error rate.
    
    Parameters
    ----------
    num_qubits : int
        Number of qubits in the surface code.
    error_rate : float
        Physical error rate for generating errors.
    
    Returns
    -------
    str
        A string representing the Pauli errors in the format "XZXZ ..."
    """
    pauli_errors = ['I', 'X', 'Y', 'Z']
    error_string = []
    
    for i in range(num_qubits):
        # Choose a random Pauli error based on the error rate
        if random.random() < error_rate:
            error = np.random.choice(pauli_errors[1:], p=[1/3, 1/3, 1/3])  # X, Y, Z
        else:
            error = 'I'
        
        error_string.append(f'{error}')
    
    return ''.join(error_string)

# Example usage
num_qubits = 24
error_rate = 0.1
error_string = generate_pauli_error_string(num_qubits, error_rate)
print("Generated error string:", error_string)

Generated error string: IIIIIIIIIIIIIIXIIIIIIYII


In [4]:
NUM_EXPERIMENTS = 10

SEED = 123
seed_seq = np.random.SeedSequence(SEED)

NUM_BITS = 4
max_bond_dims = [64, 32, 16, 8]
error_rates = np.linspace(0.000001, 0.01, 10)
failures_statistics = {}

for CHI_MAX in max_bond_dims:
    for PROB_ERROR in tqdm(error_rates):
        failures = []

        for l in range(NUM_EXPERIMENTS):
            new_seed = seed_seq.spawn(1)[0]
            rng = np.random.default_rng(new_seed)
            random_integer = rng.integers(1, 10**8 + 1)
            SEED = random_integer

            CHECK_DEGREE, BIT_DEGREE = 4, 3
            NUM_CHECKS = int(BIT_DEGREE * NUM_BITS / CHECK_DEGREE)
            if NUM_BITS / NUM_CHECKS != CHECK_DEGREE / BIT_DEGREE:
                raise ValueError("The Tanner graph of the code must be bipartite.")
            regular_code = qc.random_regular_code(
                NUM_BITS, NUM_CHECKS, BIT_DEGREE, CHECK_DEGREE, qc.Rng(SEED)
            )
            hgp_code = qc.hypergraph_product(regular_code, regular_code)

            #TODO add random seed!!
            error = generate_pauli_error_string(len(hgp_code), PROB_ERROR)
            error = pauli_to_mps(error)

            _, success = decode_css(
                code=hgp_code,
                error=error,
                chi_max=CHI_MAX,
                bias_prob=PROB_ERROR,
                renormalise=True,
                silent=True,
                contraction_strategy="Optimized",
            )

            failures.append(1 - success)

        failures_statistics[NUM_BITS, CHI_MAX, PROB_ERROR] = failures

100%|██████████| 10/10 [00:40<00:00,  4.01s/it]
